# Intro

**Author:** Stephan Cordogan

This document prepares a cohort of individuals for GWAS.  **Specify the concept IDs, or ICD and SNOMED codes, with which you would like to designate cases and controls [(a)](#5-methods-to-define-cases-and-controls-within-your-cohort,-each-increasingly-restrictive), as well as the method which you choose to designate them by [(b)](#5-methods-to-define-cases-and-controls-within-your-cohort,-each-increasingly-restrictive), and the porportion of controls you would like to use [(c)](#Specify-fraction-of-controls)**. The resultant cohort includes information on the case/control status, as well as age and sex to be used as covariates.  Meniscus tears are the example disease in this notebook, and can be substituted.

Some query logic in this notebook was adapted from the notebook 02_Hail_part1_Prepare Phenotype, with Authors: Francis Ratsimbazafy, Jennifer Zhang and Contributors: Christopher Lord, Nicole Deflaux, Kelsey Mayo, Lee Lichtenstein, CH Albach.  

In [ ]:
import os
import pandas as pd
from datetime import datetime
start = datetime.now()
bucket = os.getenv("WORKSPACE_BUCKET")
dataset = os.getenv("WORKSPACE_CDR")

# Create your cohort of WGS individuals with sex and date of birth

In [ ]:
dataset_sql = """
    SELECT
        person.person_id,
        person.birth_datetime as date_of_birth,
        p_sex_at_birth_concept.concept_name as dragen_sex_ploidy  
    FROM
        `""" + os.environ["WORKSPACE_CDR"] + """.person` person 
    LEFT JOIN
        `""" + os.environ["WORKSPACE_CDR"] + """.concept` p_sex_at_birth_concept 
            ON person.sex_at_birth_concept_id = p_sex_at_birth_concept.concept_id
    WHERE
        person.PERSON_ID IN (
            SELECT
                distinct person_id  
            FROM
                `""" + os.environ["WORKSPACE_CDR"] + """.cb_search_person` cb_search_person  
            WHERE
                cb_search_person.person_id IN (
                    SELECT
                        person_id 
                    FROM
                        `""" + os.environ["WORKSPACE_CDR"] + """.cb_search_person` p 
                    WHERE
                        has_whole_genome_variant = 1 
                ) 
            )"""

dataset_df = pd.read_gbq(
    dataset_sql,
    dialect="standard",
    use_bqstorage_api=("BIGQUERY_STORAGE_API_ENABLED" in os.environ),
    progress_bar_type="tqdm_notebook")

dataset_df.head(5)

# 5 methods to define cases and controls within your cohort, each increasingly restrictive

To execute one of the methods, ensure that the other two methods are hashtagged out and the chosen method is not.  In my experience, the more restrictive methods yield better results.  The more restrictive methods require a minimum of two separate visits, which is prescedented by Tcheandjieu et. al (linked below). Closer examination of the dataset builder tool showed that many individuals with an ICD code had condition_start_datetime/ visit_start_datetime and condition_end_datetime/ visit_end_datetime on the same day for distinct ICD codes, but distinct condition_occurrence_id's.  Individuals who had multiple distinct condition_occurrence_ids but not distinct visit_start_datetimes for those ICD code instances were far more genetically similar to individuals with distinct visit_start_datetimes than controls across a host of diseases, so method 4 is reccomended.

https://www.ncbi.nlm.nih.gov/pmc/articles/PMC9419655/

The first method defines cases and controls identically to the dataset builder tool, using concept IDs, which are broad, overarching IDs specific to a disease. This will include all of the patients with moderately associated ICD codes present. In this instance, we use the concept ID for meniscus tears and derangements. The code below is taken directly from the dataset builder tool, and simply used to define cases and controls- covariates need to be specified in the above cell. Concept IDs need to be specified in the concept_ids list above the query.

The second method, as well as all the following methods, defines cases and controls by their ICD and SNOMED codes, systems used by doctors to code medical data. The presence of an associated ICD code designates an individual as a case. ICD codes need to be specified in the immediately below cell. If there is only a single code for ICD9, 10, or SNOMED, enter it twice.

The third method defines cases and controls by their ICD and SNOMED codes, with a minimum of two instances of a code for an individual to be classified as a case. 

RECCOMENDED- The fourth method is the same as the third, but removes individuals who have one instance of the code from the GWAS, reason being it is unclear whether they truly have the disease.

The fifth method is the same as the fourth, but considers a distinct instance of the ICD code to be a distinct visit, not a distinct condition occurance ID. 

### Method 1

In [ ]:
# concept_ids = [4035415, 75346]  # Replace with your desired concept IDs

# # Convert the concept IDs to a comma-separated string
# concept_ids_str = ", ".join(map(str, concept_ids))

# dataset_haspheno_person_sql = """
#     SELECT
#         person.person_id,
#         person.gender_concept_id,
#         p_gender_concept.concept_name as gender,
#         person.birth_datetime as date_of_birth,
#         person.race_concept_id,
#         p_race_concept.concept_name as race,
#         person.ethnicity_concept_id,
#         p_ethnicity_concept.concept_name as ethnicity,
#         person.sex_at_birth_concept_id,
#         p_sex_at_birth_concept.concept_name as sex_at_birth 
#     FROM
#         `""" + os.environ["WORKSPACE_CDR"] + """.person` person 
#     LEFT JOIN
#         `""" + os.environ["WORKSPACE_CDR"] + """.concept` p_gender_concept 
#             ON person.gender_concept_id = p_gender_concept.concept_id 
#     LEFT JOIN
#         `""" + os.environ["WORKSPACE_CDR"] + """.concept` p_race_concept 
#             ON person.race_concept_id = p_race_concept.concept_id 
#     LEFT JOIN
#         `""" + os.environ["WORKSPACE_CDR"] + """.concept` p_ethnicity_concept 
#             ON person.ethnicity_concept_id = p_ethnicity_concept.concept_id 
#     LEFT JOIN
#         `""" + os.environ["WORKSPACE_CDR"] + """.concept` p_sex_at_birth_concept 
#             ON person.sex_at_birth_concept_id = p_sex_at_birth_concept.concept_id  
#     WHERE
#         person.PERSON_ID IN (SELECT
#             distinct person_id  
#         FROM
#             `""" + os.environ["WORKSPACE_CDR"] + """.cb_search_person` cb_search_person  
#         WHERE
#             cb_search_person.person_id IN (SELECT
#                 person_id 
#             FROM
#                 `""" + os.environ["WORKSPACE_CDR"] + """.cb_search_person` p 
#             WHERE
#                 has_whole_genome_variant = 1 ) 
#             AND cb_search_person.person_id IN (SELECT
#                 criteria.person_id 
#             FROM
#                 (SELECT
#                     DISTINCT person_id, entry_date, concept_id 
#                 FROM
#                     `""" + os.environ["WORKSPACE_CDR"] + """.cb_search_all_events` 
#                 WHERE
#                     (concept_id IN(SELECT
#                         DISTINCT c.concept_id 
#                     FROM
#                         `""" + os.environ["WORKSPACE_CDR"] + """.cb_criteria` c 
#                     JOIN
#                         (SELECT
#                             CAST(cr.id as string) AS id       
#                         FROM
#                             `""" + os.environ["WORKSPACE_CDR"] + """.cb_criteria` cr       
#                         WHERE
#                             concept_id IN (""" + concept_ids_str + """)         
#                             AND full_text LIKE '%_rank1]%'      ) a 
#                             ON (c.path LIKE CONCAT('%.', a.id, '.%') 
#                             OR c.path LIKE CONCAT('%.', a.id) 
#                             OR c.path LIKE CONCAT(a.id, '.%') 
#                             OR c.path = a.id) 
#                     WHERE
#                         is_standard = 1 
#                         AND is_selectable = 1) 
#                     AND is_standard = 1 )) criteria ) )"""

# dataset_haspheno_person_df = pd.read_gbq(
#     dataset_haspheno_person_sql,
#     dialect="standard",
#     use_bqstorage_api=("BIGQUERY_STORAGE_API_ENABLED" in os.environ),
#     progress_bar_type="tqdm_notebook")

# # Ensure 'person_id' is the index in dataset_haspheno_person_df
# dataset_haspheno_person_ids = set(dataset_haspheno_person_df['person_id'])

# # Create a new column 'has_pheno' in dataset_pheno_df based on the presence in dataset_haspheno_person_ids

# dataset_df['has_pheno'] = dataset_df['person_id'].isin(dataset_haspheno_person_ids).astype(int)

# # The result is final_cohort with has_pheno set to 1 if person_id is in dataset_haspheno_person_df, otherwise is 0

# final_cohort = dataset_df

#### Required for methods 2-5

In [ ]:
condition_codes_icd9 = tuple(['717.0', '717.1', '717.2', '717.3', '717.4', '717.5','836.0', '836.1'])
condition_codes_icd10 = tuple([ 
                                  "M23.2", "M23.20", "M23.200", "M23.201", "M23.202", "M23.203", "M23.204", 
                                  "M23.205", "M23.206", "M23.207", "M23.209", 
                                  "M23.22", "M23.211", "M23.212", "M23.219", 
                                  "M23.22", "M23.221", "M23.222", "M23.229", 
                                  "M23.23", "M23.231", "M23.232", "M23.239", 
                                  "M23.24", "M23.241", "M23.242", "M23.249", 
                                  "M23.25", "M23.251", "M23.252", "M23.259", 
                                  "M23.26","M23.261", "M23.262", "M23.269", 
                                  "M23.3", "M23.30","M23.300", "M23.301", "M23.303", "M23.302", "M23.304", "M23.305", 
                                  "M23.306", "M23.307", "M23.309", 
                                  "M23.31", "M23.311", "M23.312", "M23.319", 
                                  "M23.32", "M23.321", "M23.322", "M23.329", 
                                  "M23.33", "M23.331", "M23.332", "M23.339",
                                  "M23.34", "M23.341", "M23.342", "M23.349", 
                                  "M23.35", "M23.351", "M23.352", "M23.359", 
                                  "M23.36", "M23.361", "M23.362", "M23.369", 
                                  "S83.2", "S83.20", "S83.200A", "S83.201A", "S83.202A", "S83.203A", "S83.204A", "S83.205A", 
                                  "S83.206A", "S83.207A", "S83.209A", "S83.211A", "S83.212A", "S83.219A", 
                                  "S83.221A", "S83.222A", "S83.229A", "S83.231A", "S83.232A", "S83.239A", "S83.241A", 
                                  "S83.242A", "S83.249A", "S83.251A", "S83.252A", "S83.259A", "S83.261A", "S83.262A", 
                                  "S83.269A", "S83.271A", "S83.272A", "S83.279A", "S83.281A", "S83.282A", "S83.289A", 
    "S83.200D", "S83.201D", "S83.202D", "S83.203D", "S83.204D", "S83.205D", "S83.206D", 
    "S83.207D", "S83.209D", "S83.211D", "S83.212D", "S83.219D", "S83.221D", 
    "S83.222D", "S83.229D", "S83.231D", "S83.232D", "S83.239D", "S83.241D", "S83.242D", 
    "S83.249D", "S83.251D", "S83.252D", "S83.259D", "S83.261D", "S83.262D", "S83.269D", 
    "S83.271D", "S83.272D", "S83.279D", "S83.281D", "S83.282D", "S83.289D", "S83.200S", 
    "S83.201S", "S83.202S", "S83.203S", "S83.204S", "S83.205S", "S83.206S", "S83.207S", 
    "S83.209S", "S83.211S", "S83.212S", "S83.219S", "S83.221S", "S83.222S", 
    "S83.229S", "S83.231S", "S83.232S", "S83.239S", "S83.241S", "S83.242S", "S83.249S", 
    "S83.251S", "S83.252S", "S83.259S", "S83.261S", "S83.262S", "S83.269S", "S83.271S", 
    "S83.272S", "S83.279S", "S83.281S", "S83.282S", "S83.289S"])
                                
condition_codes_SNOMED = tuple(['2397200000', '302933001', '302932006', '111224002', '213330004', '5313005','77860008'
                    ,'86378006', '11859741000119109', '15637431000119106', '11859701000119107', 
                               '15637391000119101', '296201000119105', '11859661000119100' ])


### Method 2

In [ ]:
# # This code retrieves concept ID for conditions matching those codes, defining the query

# query = f"""                                
# SELECT 
#     c.concept_id
# FROM `{dataset}.concept` c
# JOIN `{dataset}.condition_occurrence` co ON c.concept_id = co.condition_source_concept_id
# WHERE (vocabulary_id='ICD9CM' AND concept_code IN {condition_codes_icd9})
#     OR (vocabulary_id='ICD10CM' AND concept_code IN {condition_codes_icd10}) 
#     OR (vocabulary_id='SNOMED' AND concept_code IN {condition_codes_SNOMED})

# GROUP BY c.concept_name, c.concept_code,c.concept_id
# """

# # This code executes the query

# condition_concepts_df  = pd.read_gbq(query, dialect = "standard")

# # This code retrieves unique person ID from person table for those who have condition codes above and adds an indicator variable has_ocdc for them

# query = f"""                                
# SELECT person.person_id, 
#    -- Add an indicator variable.
#     1 AS has_pheno
# FROM `{dataset}.person` person
# WHERE
#     person_id IN (SELECT person_id
#                   FROM `{dataset}.condition_occurrence`
#                   WHERE condition_source_concept_id IN {tuple(condition_concepts_df['concept_id'])})
# """
# cohort = pd.read_gbq(query, dialect="standard")

# final_cohort = (dataset_df.merge(cohort, on='person_id', how='left')
#               .fillna(value={'has_pheno': 0})
#              )
# final_cohort['has_pheno'].value_counts()

### Method 3

In [ ]:
# # This code retrieves concept ID for conditions matching those codes, defining the query 
# # Individuals included must have 2+ instances of a code

# query = f"""
# SELECT 
#     co.person_id,
#     COUNT(DISTINCT co.condition_occurrence_id) AS occurrence_count
# FROM `{dataset}.concept` c
# JOIN `{dataset}.condition_occurrence` co 
#   ON c.concept_id = co.condition_source_concept_id
# WHERE 
#     (vocabulary_id='ICD9CM' AND concept_code IN {condition_codes_icd9})
#     OR (vocabulary_id='ICD10CM' AND concept_code IN {condition_codes_icd10}) 
#     OR (vocabulary_id='SNOMED' AND concept_code IN {condition_codes_SNOMED})
# GROUP BY co.person_id
# HAVING COUNT(DISTINCT co.condition_occurrence_id) > 1
# """

# # This code executes the query

# condition_concepts_df = pd.read_gbq(query, dialect="standard")

# # This code retrieves unique person ID from person table for those who have condition codes above and adds an indicator variable has_ocdc for them

# person_ids = tuple(condition_concepts_df['person_id'].unique())

# query = f"""
# SELECT person.person_id, 
#    -- Add an indicator variable.
#     1 AS has_pheno
# FROM `{dataset}.person` person
# WHERE person_id IN {person_ids}
# """

# cohort = pd.read_gbq(query, dialect="standard")

# final_cohort = (dataset_df.merge(cohort, on='person_id', how='left')
#               .fillna(value={'has_pheno': 0})
#              )
# final_cohort['has_pheno'].value_counts()

# #6262 2382 

In [ ]:
# print(final_cohort.columns)


### Method 4

In [ ]:
# This code retrieves concept ID for conditions matching those codes, defining the query 
# Individuals included must have 2+ instances of a code

query_2plus = f"""
SELECT 
    co.person_id,
    COUNT(DISTINCT co.condition_occurrence_id) AS occurrence_count
FROM `{dataset}.concept` c
JOIN `{dataset}.condition_occurrence` co 
  ON c.concept_id = co.condition_source_concept_id
WHERE 
    (vocabulary_id='ICD9CM' AND concept_code IN {condition_codes_icd9})
    OR (vocabulary_id='ICD10CM' AND concept_code IN {condition_codes_icd10}) 
    OR (vocabulary_id='SNOMED' AND concept_code IN {condition_codes_SNOMED})
GROUP BY co.person_id
HAVING COUNT(DISTINCT co.condition_occurrence_id) > 1
"""
query1 = f"""
SELECT 
    co.person_id,
    COUNT(DISTINCT co.condition_occurrence_id) AS occurrence_count
FROM `{dataset}.concept` c
JOIN `{dataset}.condition_occurrence` co 
  ON c.concept_id = co.condition_source_concept_id
WHERE 
    (vocabulary_id='ICD9CM' AND concept_code IN {condition_codes_icd9})
    OR (vocabulary_id='ICD10CM' AND concept_code IN {condition_codes_icd10}) 
    OR (vocabulary_id='SNOMED' AND concept_code IN {condition_codes_SNOMED})
GROUP BY co.person_id
HAVING COUNT(DISTINCT co.condition_occurrence_id) = 1
"""

condition_concepts_2plus_df = pd.read_gbq(query_2plus, dialect="standard")
condition_concepts_1only_df = pd.read_gbq(query1, dialect="standard")

person_ids = tuple(condition_concepts_2plus_df['person_id'].unique())
person_ids1 = tuple(condition_concepts_1only_df['person_id'].unique())

query = f"""
SELECT person.person_id, 
   -- Add an indicator variable.
    CASE
        WHEN person_id IN {person_ids1} THEN 2  -- Assign 2 if in person_ids1
        WHEN person_id IN {person_ids} THEN 1   -- Assign 1 if in person_ids
        ELSE 0  -- Optional: Assign 0 if not in either
    END AS has_pheno
FROM `{dataset}.person` person
"""

cohort = pd.read_gbq(query, dialect="standard")

final_cohort = (dataset_df.merge(cohort, on='person_id', how='left')
              .fillna(value={'has_pheno': 0})
             )
final_cohort['has_pheno'].value_counts()

final_cohort = final_cohort[final_cohort['has_pheno'] != 2]
final_cohort['has_pheno'].value_counts()


### Method 5

In [ ]:
# # This code retrieves concept ID for conditions matching those codes, defining the query 
# # Individuals included must have 2+ instances of a code

# query_2plus = f"""
# SELECT 
#     co.person_id,
#     COUNT(DISTINCT co.condition_occurrence_id) AS occurrence_count,
#     COUNT(DISTINCT vo.visit_start_date) AS distinct_visit_count
# FROM `{dataset}.concept` c
# JOIN `{dataset}.condition_occurrence` co 
#   ON c.concept_id = co.condition_source_concept_id
# JOIN `{dataset}.visit_occurrence` vo
#   ON co.visit_occurrence_id = vo.visit_occurrence_id
# WHERE 
#     (vocabulary_id='ICD9CM' AND concept_code IN {condition_codes_icd9})
#     OR (vocabulary_id='ICD10CM' AND concept_code IN {condition_codes_icd10}) 
#     OR (vocabulary_id='SNOMED' AND concept_code IN {condition_codes_SNOMED})
# GROUP BY co.person_id
# HAVING COUNT(DISTINCT co.condition_occurrence_id) > 1
#    AND COUNT(DISTINCT vo.visit_start_date) > 1
# """

# query1 = f"""
# SELECT 
#     co.person_id,
#     COUNT(DISTINCT co.condition_occurrence_id) AS occurrence_count,
#     COUNT(DISTINCT vo.visit_start_date) AS distinct_visit_count
# FROM `{dataset}.concept` c
# JOIN `{dataset}.condition_occurrence` co 
#   ON c.concept_id = co.condition_source_concept_id
# JOIN `{dataset}.visit_occurrence` vo
#   ON co.visit_occurrence_id = vo.visit_occurrence_id
# WHERE 
#     (vocabulary_id='ICD9CM' AND concept_code IN {condition_codes_icd9})
#     OR (vocabulary_id='ICD10CM' AND concept_code IN {condition_codes_icd10}) 
#     OR (vocabulary_id='SNOMED' AND concept_code IN {condition_codes_SNOMED})
# GROUP BY co.person_id
# HAVING 
#     (COUNT(DISTINCT co.condition_occurrence_id) = 1)
#     OR (COUNT(DISTINCT co.condition_occurrence_id) > 1 AND COUNT(DISTINCT vo.visit_start_date) = 1)
# """

# condition_concepts_2plus_df = pd.read_gbq(query_2plus, dialect="standard")
# condition_concepts_1only_df = pd.read_gbq(query1, dialect="standard")

# person_ids = tuple(condition_concepts_2plus_df['person_id'].unique())
# person_ids1 = tuple(condition_concepts_1only_df['person_id'].unique())

# query = f"""
# SELECT person.person_id, 
#    -- Add an indicator variable.
#     CASE
#         WHEN person_id IN {person_ids1} THEN 2  -- Assign 2 if in person_ids1
#         WHEN person_id IN {person_ids} THEN 1   -- Assign 1 if in person_ids
#         ELSE 0  -- Optional: Assign 0 if not in either
#     END AS has_pheno
# FROM `{dataset}.person` person
# """

# cohort = pd.read_gbq(query, dialect="standard")

# final_cohort = (dataset_df.merge(cohort, on='person_id', how='left')
#               .fillna(value={'has_pheno': 0})
#              )
# final_cohort['has_pheno'].value_counts()

# final_cohort = final_cohort[final_cohort['has_pheno'] != 2]
# final_cohort['has_pheno'].value_counts()

# Process and filter data, save to workspace bucket

This code converts age into a continuous variable

In [ ]:
current_date = pd.Timestamp('now', tz='UTC')
final_cohort['age'] = current_date-final_cohort.date_of_birth
final_cohort['age_yrs'] = final_cohort.age/pd.Timedelta('365.25 days')
final_cohort.head(5)

In [ ]:
demographics = pd.get_dummies(final_cohort.set_index(['person_id'])).reset_index()
demographics['has_pheno'] = demographics['has_pheno'].astype(int)
demographics.head()

# Specify fraction of controls
The code below specifies the porportion of the total controls to be included in the GWAS.  This should be ~4x larger than your cases if possible.  There are no consequences to a much larger control cohort other than the increased computational costs, and your results will be more reproducible. Set the porportion of overall controls in the GWAS with frac= (frac=1 includes 100% of overall controls).

In [ ]:
df_has_pheno_1 = demographics[demographics['has_pheno'] == 1]
df_has_pheno_0 = demographics[demographics['has_pheno'] == 0].sample(frac=1, random_state=1)
demographics_reduced = pd.concat([df_has_pheno_1, df_has_pheno_0]).reset_index(drop=True)

In [ ]:
females = demographics_reduced['dragen_sex_ploidy_Female'].sum()
males = demographics_reduced['dragen_sex_ploidy_Male'].sum()
total = len(demographics_reduced)
print(f"Females: {females}, Males: {males}, Total: {total}")

In [ ]:
# Number of rows in df_reduced
num_rows = len(demographics_reduced)
print(f"Number of rows in df_reduced: {num_rows}")

The first line below removes individuals not classified as females or males

In [ ]:
filtered_data = demographics_reduced[
    (demographics_reduced['dragen_sex_ploidy_Female'] != demographics_reduced['dragen_sex_ploidy_Male'])
]

# Select and rename the columns
phenotypes = (filtered_data[["person_id", "has_pheno", "dragen_sex_ploidy_Female", "dragen_sex_ploidy_Male", "age_yrs"]]
              .rename(columns={'dragen_sex_ploidy_Female': 'is_female', 'dragen_sex_ploidy_Male': 'is_male'})
             )
phenotypes['is_male'] = phenotypes['is_male'].astype(int)

num_rows = len(phenotypes)
print(f"Number of rows in phenotypes: {num_rows}")

In [ ]:
phenotypes["person_id"] = phenotypes["person_id"].astype(str)
    
phenotypes.to_csv('genomics_phenotypes.tsv', index=False, sep='\t')

# save phenotypes to the bucket
!gsutil cp 'genomics_phenotypes.tsv' {bucket}/data/

In [ ]:
phenotypes['has_pheno'].value_counts()

In [ ]:
!gsutil -m ls {bucket}/data/